<a href="https://colab.research.google.com/github/rajibmondal/nlp_course/blob/master/seq2seq_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import drive

In [0]:
from fastai.text import *

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
 folder = 'NLP_DATA'##for create 'NLP_DATA' folder into this "/content/gdrive/My Drive/NLP/NLP_DATA"
# file = 'eng-fra2.csv'

In [0]:
from fastai.text import *

path = Path('/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [14]:
path.ls()

[PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq/eng-fra.csv'),
 PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq/NLP_DATA'),
 PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq/data_save.pkl')]

In [0]:
import pandas as pd


In [0]:
df = pd.read_csv(path/'eng-fra.csv')

In [17]:
df.head()

,en,fr
0,Run!,Cours !
1,Run!,Courez !
2,Wow!,Ça alors !
3,Fire!,Au feu !
4,Help!,À l'aide !


In [18]:
df

,en,fr
0,Run!,Cours !
1,Run!,Courez !
2,Wow!,Ça alors !
3,Fire!,Au feu !
4,Help!,À l'aide !
...,...,...
135836,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
135837,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
135838,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
135839,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [0]:
df['en'] = df['en'].apply(lambda x:x.lower())
df['fr'] = df['fr'].apply(lambda x:x.lower())

In [20]:
df

,en,fr
0,run!,cours !
1,run!,courez !
2,wow!,ça alors !
3,fire!,au feu !
4,help!,à l'aide !
...,...,...
135836,a carbon footprint is the amount of carbon dio...,une empreinte carbone est la somme de pollutio...
135837,death is something that we're often discourage...,la mort est une chose qu'on nous décourage sou...
135838,since there are usually multiple websites on a...,puisqu'il y a de multiples sites web sur chaqu...
135839,if someone who doesn't know your background sa...,si quelqu'un qui ne connaît pas vos antécédent...


In [0]:
def seq2seq_collate(samples, pad_idx=1, pad_first=True, backwards=False):
  "Function that collect samples and add padding.Flips token in order to needed"
  samples = to_data(samples)
  max_len_x, max_len_y = max([len(s[0]) for s in samples]), max([len(s[1]) for s in samples])
  res_x = torch.zeros(len(samples), max_len_x).long() + pad_idx
  res_y = torch.zeros(len(samples), max_len_y).long() + pad_idx
  if backwards: pad_first = not pad_first

  for i, s in enumerate(samples):
    if pad_first:
      res_x[i, -len(s[0]):], res_y[i, -len(s[1]):] = LongTensor(s[0]), LongTensor(s[1])
    else:
      res_x[i, :len(s[0]):], res_y[i, :len(s[1]):] = LongTensor(s[0]), LongTensor(s[1])
  if backwards: res_x, res_y = res_x.flip(1), res_y.flip(1)
  return res_x,res_y


In [0]:
class Seq2SeqDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training an RNN classifier."
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=32, val_bs:int=None, pad_idx=1,
               dl_tfms=None, pad_first=False, device:torch.device=None, no_check:bool=False, backwards:bool=False, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(seq2seq_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, collate_fn=collate_fn, no_check=no_check)

In [0]:
class Seq2SeqTextList(TextList):
  _bunch = Seq2SeqDataBunch
  _label_cls = TextList

In [24]:
src = Seq2SeqTextList.from_df(df, path=path, cols='fr').split_by_rand_pct(seed=42).label_from_df(cols='en', label_cls=TextList)

In [25]:
np.percentile([len(o) for o in src.train.x.items] + [len(o) for o in src.valid.x.items], 90)

13.0

In [26]:
np.percentile([len(o) for o in src.train.y.items] + [len(o) for o in src.valid.y.items], 90)

12.0

In [0]:
src = src.filter_by_func(lambda x,y: len(x)>30 or len(y) > 30)

In [28]:
len(src.train) + len(src.valid)

135790

In [0]:
data = src.databunch()


In [0]:
data.save()

In [0]:
# model_path = Config().model_path

In [0]:
# torch.save(data, model_path/'data.pth')

In [31]:
data.path

PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq')

In [32]:
path

PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq')

In [33]:
data

Seq2SeqDataBunch;

Train: LabelList (108635 items)
x: Seq2SeqTextList
xxbos courez   !,xxbos ça alors   !,xxbos au feu !,xxbos saute .,xxbos ça suffit   !
y: TextList
xxbos run !,xxbos wow !,xxbos fire !,xxbos jump .,xxbos stop !
Path: /content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq;

Valid: LabelList (27155 items)
x: Seq2SeqTextList
xxbos elle nous a cuisiné un déjeuner délicieux .,xxbos aimes - tu être seul ?,xxbos j'ai obtenu ce que j'ai demandé .,xxbos arrête juste ici !,xxbos tu es l'élu .
y: TextList
xxbos she cooked us a delicious dinner .,xxbos do you like to be alone ?,xxbos i got what i asked for .,xxbos stop right here .,xxbos you are the one .
Path: /content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq;

Test: None

In [0]:
data = load_data(path)

In [35]:
data.show_batch()

text,target
"xxbos une bonne théorie se xxunk par le fait de faire une série de xxunk qui , en principe , pourraient être xxunk ou mises en défaut par l'observation .",xxbos a good theory is xxunk by the fact that it makes a number of xxunk that could in principle be xxunk or xxunk by xxunk .
"xxbos quelqu'un m'a dit xxunk xxunk déclara : "" le bon sens est la collection des préjugés acquis à l'âge de dix - huit ans . ""","xxbos someone told me that xxunk xxunk said , "" common sense is the collection of xxunk acquired by age eighteen . """
xxbos y a - t - il quoi que ce soit que tu veuilles ajouter à ce que je viens de dire ?,xxbos is there anything you want to add to what i just said ?
xxbos xxunk % de ceux qui n'ont jamais fumé ont dit qu'ils xxunk des effets sur la santé de la fumée passive .,xxbos xxunk % of those who never smoked said they worried about the health effects of passive smoking .
"xxbos lorsque nous arrivâmes chez nous hier au soir , les autres étaient déjà au lit et dormaient à poings fermés .","xxbos when we got home last night , the others were already in bed and fast asleep ."


#### Pretrained Embedding

In [1]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 3653 (delta 53), reused 67 (delta 27), pack-reused 3531
Receiving objects: 100% (3653/3653), 8.16 MiB | 32.77 MiB/s, done.
Resolving deltas: 100% (2278/2278), done.


In [0]:
!cd fastText

In [3]:
!pip install .

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [0]:
import fastText as ft

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My\ Drive/"
base_dir = root_dir + 'fastai-v3/'



Mounted at /content/gdrive


In [23]:
from fastai.text import *
path = Path('/content/gdrive/My \Drive/NLP/NLP_DATA/Seq2Seq')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

OSError: ignored

In [7]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}

--2020-01-23 15:05:52--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
/content/gdrive/My: Operation not supported
/content/gdrive/My/cc.en.300.bin.gz: No such file or directory

Cannot write to ‘/content/gdrive/My/cc.en.300.bin.gz’ (Success).
--2020-01-23 15:05:53--  http://drive/NLP/NLP_DATA/Seq2Seq
Resolving drive (drive)... failed: No address associated with hostname.
wget: unable to resolve host address ‘drive’


In [37]:
gunzip {path} / cc.en.300.bin.gz

SyntaxError: ignored

In [38]:
en_vecs = ft.load_model(str((path/'cc.en.300.bin')))

AttributeError: ignored

In [40]:
from pyfasttext import FastText
model = FastText('/home/sproc/share/fastText/model.bin')

ModuleNotFoundError: ignored

In [41]:
import FastText

ModuleNotFoundError: ignored

In [42]:
!pip install FastText

     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for FastText: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2390352 sha256=310642457c90cbee0114542ffdfacf68a27b421563e5c6643d58d88217699160
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built FastText


In [45]:
!git clone https://github.com/facebookresearch/fastText.git

fatal: destination path 'fastText' already exists and is not an empty directory.


In [46]:
cd fastText

/content/fastText


In [47]:
pip install .

Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2861853 sha256=d6cbad4c64bdba66086a82b3516ce7bc9d8395abb4ef81fe60d8073b00e1e819
  Stored in directory: /tmp/pip-ephem-wheel-cache-23skqovj/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
  Found existing installation: fasttext 0.9.1
    Uninstalling fasttext-0.9.1:
      Successfully uninstalled fasttext-0.9.1


In [0]:
import fastText

In [0]:
from gensim.models import FastText

In [53]:
# Original fasttext embeddings from https://fasttext.cc/
ft = FastText.load_fasttext_format("fasttext_embedding.bin")

FileNotFoundError: ignored

In [0]:
path

PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq')

In [0]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz -P {path}

--2020-01-08 09:11:27--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
/content/gdrive/My: Operation not supported
/content/gdrive/My/cc.en.300.bin.gz: No such file or directory

Cannot write to ‘/content/gdrive/My/cc.en.300.bin.gz’ (Success).
--2020-01-08 09:11:28--  http://drive/NLP/NLP_DATA/Seq2Seq
Resolving drive (drive)... failed: No address associated with hostname.
wget: unable to resolve host address ‘drive’
--2020-01-08 09:11:30--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublic

In [0]:
gunzip {path} / cc.en.300.bin.gz
gunzip {path} / cc.fr.300.bin.gz

In [0]:
#gunzip {'/content/gdrive/My Drive/NLP/NLP_DATA/Seq2Seq'} / cc.en.300.bin.gz
 gunzip path / cc.fr.300.bin.gz

In [0]:
fr_vecs = ft.load_model(str((path/'cc.fr.300.bin')))
en_vecs = ft.load_model(str((path/'cc.en.300.bin')))

In [0]:
 ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}

--2020-01-08 09:21:20--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
/content/gdrive/My: Operation not supported
/content/gdrive/My/cc.en.300.bin.gz: No such file or directory

Cannot write to ‘/content/gdrive/My/cc.en.300.bin.gz’ (Success).
--2020-01-08 09:21:21--  http://drive/NLP/NLP_DATA/Seq2Seq
Resolving drive (drive)... failed: No address associated with hostname.
wget: unable to resolve host address ‘drive’


In [0]:
from gensim.models import FastText

#ft_cc = FastText.load_fasttext_format("models/cc.en.300.bin.gz"#, full_model=False)

In [0]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz -P {path}

In [0]:
gunzip {path} / cc.en.300.bin.gz
gunzip {path} / cc.fr.300.bin.gz

In [0]:
fr_vecs = ft.load_model(str((path/'cc.fr.300.bin')))
en_vecs = ft.load_model(str((path/'cc.en.300.bin')))

In [0]:
def create_emb(vecs, itos,em_sz=300,mult=1.):
  emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
  wgts = emb.weight.data
  vec_dic = {w:vecs.get_word_vector(w) for w in vecs.get_words()}
  miss=[]
  for i,w in enumerate(itos):
    try:wgts[i] = tensor(vec_dic[w])
    except: miss.append(w)
  return emb

In [0]:
class Seq2Seq(nn.Module):
  def __init__(self, emb_enc, emb_dec, nh, out_sl, n1=2,
               bos_idx=0, pad_idx=1):
    super().__init__()
    self.nl, self.nh,self.out_sl = nl, nh, out_sl
    self.bos_idx, self.pad_idx = bos_idx, pad_idx
    self.em_sz_enc = emb_enc.embedding_dim
    self.em_sz_dec = emb_dec.embedding_dim
    self.voc_sz_dec = emb_dec.num_embeddings

    self.emb_enc = emb_enc
    self.emb_enc_drop = nn.Dropout(0.15)
    self.gru_enc = nn.GRU(self.em_sz_enc, nh, num_layers=nl,
                          dropout=0.25, batch_first=True)
    self.out_enc = nn.Linear(nh, self.em_sz_dec,bias=False)

    self.emb_dec = emb_dec
    self.gru_dec = nn.GRU(self.em_sz_dec, num_layers=nl,
                          dropout=0.1, batch_first=True)
    self.out_drop = nn.Dropout(0.35)
    self.out = nn.Linear(self.em_sz_dec, self.voc_sz_dec)
    self.out.weight.data = self.emb_dec.weight.data

  def encoder(self, bs, inp):
    h = self.initHidden(bs)
    emb = self.emb_enc_drop(self.emb_enc(inp))
    _,h = self.gru_enc(emb,h)
    h = self.out_enc(h)
    return h

  def decoder(self, dec_inp, h):
    emb = self.emb_dec(dec_inp).unsqueeze(1)
    outp, h = self.gru_dec(emb, h)
    outp = self.out(self.out_drop(outp[:,0]))
    return h, outp


    


